In [1]:
print("hi")

hi


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import math

print("Loading Gemma3 4B model...")
model_path = "/scratch/rohank__iitp/gemma3_4b_it"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16)
model.eval()

In [4]:
def calculate_perplexity(text: str) -> float:
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    input_ids = inputs["input_ids"]

    with torch.no_grad():
        outputs = model(input_ids=input_ids, labels=input_ids)
        loss = outputs.loss

    return math.exp(loss.item())


In [7]:
calculate_perplexity("H. The weather is perfect for a walk in the park. I agree, it's a great day to be outside.")

12.889538775015744

In [8]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def compute_bleu2(candidate_sentence: str, reference_sentence: str) -> float:
    """
    Computes BLEU-2 score (unigram + bigram with equal weights) between a candidate and reference sentence.

    Args:
        candidate_sentence (str): The generated sentence.
        reference_sentence (str): The reference (ground truth) sentence.

    Returns:
        float: BLEU-2 score (between 0 and 1)
    """
    # Tokenize
    candidate_tokens = candidate_sentence.strip().split()
    reference_tokens = reference_sentence.strip().split()
    
    # BLEU-2 with weights (0.5, 0.5) → unigram and bigram equally
    weights = (0.5, 0.5)
    
    # Optional smoothing to avoid zero scores for short or imperfect matches
    smoothing = SmoothingFunction().method1

    # Compute score
    bleu2_score = sentence_bleu([reference_tokens], candidate_tokens, weights=weights, smoothing_function=smoothing)
    
    return bleu2_score


cand = "the cat is on the mat"
ref = "there is a cat on the mat"

score = compute_bleu2(cand, ref)
print(f"BLEU-2 Score: {score:.4f}")


BLEU-2 Score: 0.4887


In [27]:

from bert_score import score

def compute_bert_score_f1(candidates, references, lang="en", model_type="bert-base-uncased", verbose=False):

    P, R, F1 = score(candidates, references, lang=lang, model_type=model_type, verbose=verbose)
    x=F1.tolist()
    return float(x[0])   # Convert torch.Tensor to list for usability



In [31]:

candidates = ["The cat is on the mat"]
references = ["Therat"]

bs_f1_scores = compute_bert_score_f1(candidates, references)
# print(f"BERTScore-F1: {bs_f1_scores[0]:.4f}")
type(bs_f1_scores)
round(bs_f1_scores,3)

0.565

In [17]:
from collections import Counter

def distinct_2(text):

    words = text.strip().split()
    if len(words) < 2:
        return 0.0  # No bigrams can be formed

    bigrams = [(words[i], words[i+1]) for i in range(len(words) - 1)]
    total_bigrams = len(bigrams)
    unique_bigrams = len(set(bigrams))

    return unique_bigrams / total_bigrams

# Example usage
example_text = "I like cat like cat I like cat"
print("Distinct-2 score:", distinct_2(example_text))


Distinct-2 score: 0.5714285714285714


In [32]:
from rouge_score import rouge_scorer

def calculate_rouge1(reference, candidate):

    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return scores['rouge1'].fmeasure  # returns a namedtuple with precision, recall, fmeasure

# Example usage:
ref = "The cat sat on the mat."
cand = "The cat is sitting on the mat."
result = calculate_rouge1(ref, cand)
print(f"ROUGE-1 ->: {result}")
type(result)

ROUGE-1 ->: 0.7692307692307692


float

In [ ]:
import pandas as pd

# Load your dataset
df = pd.read_csv("your_csv_file.csv")

# Initialize empty columns
df['PPL'] = 0.0
df['BLEU-2'] = 0.0
df['BERTScore-F1'] = 0.0
df['Distinct-2'] = 0.0
df['ROUGE-1'] = 0.0

# Process each row and update CSV after each change
for i in range(len(df)):
    agent_reply = str(df.loc[i, 'new_agent_reply'])
    model_reply = str(df.loc[i, 'Gemma Allexp Response'])

    df.loc[i, 'PPL'] = calculate_perplexity(model_reply)
    df.loc[i, 'BLEU-2'] = compute_bleu2(model_reply, agent_reply)
    df.loc[i, 'BERTScore-F1'] = compute_bert_score_f1([model_reply], [agent_reply])
    df.loc[i, 'Distinct-2'] = distinct_2(model_reply)
    df.loc[i, 'ROUGE-1'] = calculate_rouge1(agent_reply, model_reply)

    # Write to CSV after every row
    df.to_csv("live_updated_file.csv", index=False)
